In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from TX import bcc_rolling_fiber
import time

- Create fiber textures in various intensity levels
by applying different weights of $\gamma$-fiber ($\chi^\chi$) and random textures ($\chi^\text{random}$).

$\chi^\text{total}$=$f \chi^\gamma$+$(1-f)\chi^\text{random}$

In [3]:
%%time
polyXtals=[]
fraction = np.linspace(0,1,100)
for i in xrange(len(fraction)):
    gammaXtal=bcc_rolling_fiber.main(
        ngrains=500,
        fiber='gamma',
        sigma=15,iexit=True)
    frac1 = fraction[i]
    frac2 = 1.-frac1
    randomXtal=bcc_rolling_fiber.main(
        ngrains=500,
        fiber='random',
        iexit=True)
    gammaXtal[:,-1]=gammaXtal[:,-1]*frac1
    randomXtal[:,-1]=randomXtal[:,-1]*frac2
    pxtal = np.concatenate((gammaXtal,randomXtal),axis=0)
    polyXtals.append(pxtal)

CPU times: user 19.7 s, sys: 159 ms, total: 19.9 s
Wall time: 20 s


- Plot pole figures

In [4]:
from TX import upf ## load pole figure plotter

- Plot (111) pole figure and save them to /tmp/pf_???.png files

In [11]:
%%time
for i in xrange(100):
    mypf = upf.polefigure(grains=polyXtals[i],csym='cubic')
    fig=mypf.pf_new(poles=[[1,1,1]],cmap='jet',nlev=7,mn=0.8,mx=10)
    fig.savefig('/tmp/pf_%3.3i.png'%i,bbox_inches='tight',dpi=300)
    fig.clf()
    plt.close(fig)

CPU times: user 2min 41s, sys: 4.03 s, total: 2min 45s
Wall time: 2min 56s


- Convert pngs to mp4 using imagemagick

In [6]:
import os
from glob import glob
fns=glob('/tmp/pf_???.png')
cmd ='convert -delay 8 -quality 100'
for i in xrange(len(fns)):
    cmd = '%s %s'%(cmd, fns[i])
cmd = '%s %s'%(cmd, 'GammaFibAni.mpg')
os.system(cmd)

0

- Display the video

In [7]:
import io
import base64
from IPython.display import HTML

video = io.open('gammaFibAni.mpg', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
             <source src="data:video/mpg;base64,{0}" type="video/mpg" />
             </video>'''.format(encoded.decode('ascii')))

- Convert png files to a GIF movie file

In [18]:
## If there's an existing file
!rm gammaFibAni.gif

In [19]:
!ffmpeg -i /tmp/pf_%03d.png gammaFibAni.gif

ffmpeg version 3.0.2 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 7.3.0 (clang-703.0.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.0.2 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --enable-libfaac --enable-nonfree --enable-vda
  libavutil      55. 17.103 / 55. 17.103
  libavcodec     57. 24.102 / 57. 24.102
  libavformat    57. 25.100 / 57. 25.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 31.100 /  6. 31.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libswresample   2.  0.101 /  2.  0.101
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '/tmp/pf_%03d.png':
  Duration: 00:00:04.00, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 1051x861 [SAR 11811:11811 DAR 1051:861], 25 f